In [21]:
import os
import sys
root_dir = os.path.split(os.getcwd())[0]
project_dir = os.path.join(root_dir, 'python')
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [22]:
import subprocess
output = subprocess.Popen(f"cd {project_dir} && pipenv --venv", shell=True, stdout=subprocess.PIPE).stdout.read().decode('utf-8')
if len(output) > 0:
    # Include the pipenv in this notebook
    pipenv_path = os.path.join(output[:-1], 'lib', 'python3.6', 'site-packages')
    if pipenv_path not in sys.path:
        sys.path.append(pipenv_path)

In [27]:
url = "http://localhost:9090"
user = "admin"
password = "admin"

In [28]:
from pivot import Connector
from datetime import datetime

connector = Connector(url, user, password)

In [29]:
stores = connector.stores()
print(f"stores: {stores}")
fields = connector.store_fields('Risk')
print(f'fields: {fields}')
refs = connector.store_references('Risk')
print(f'refs {refs}')
print(connector.store_fields('Trade'))

stores: ['City', 'CounterParty', 'Forex', 'Parameters', 'Product', 'Risk', 'Trade']
fields: {'TradeId': None, 'AsOfDate': None, 'HostName': <class 'str'>, 'delta': None, 'pnlDelta': None, 'gamma': None, 'vega': None, 'pnlVega': None, 'pnl': None}
refs {'RiskToTrade': {'targetStore': 'Trade', 'fieldMappings': {'TradeId': 'Id'}}}
{'Id': None, 'ProductId': <class 'int'>, 'ProductQtyMultiplier': None, 'Desk': <class 'str'>, 'BookId': None, 'CounterParty': <class 'str'>, 'Date': None, 'DateBucket': None, 'Status': <class 'str'>, 'IsSimulated': <class 'str'>}


In [ ]:
# query_ds = connector.store_query('Risk', fields = ['AsOfDate', 'RiskToTrade/CounterParty', 'RiskToTrade/Desk'])
# Result by a datastore query
current_date = datetime.now()
target_date = f"{current_date.year}-{current_date.month}-{current_date.day}"

query_ds = connector.store_query('Risk', fields = ['AsOfDate', 'pnl', 'vega'])

In [32]:
# print(f"ds query = {query_ds.dataframe.describe()}")
aggr_ds_query = query_ds.dataframe.groupby('AsOfDate').sum()
print(f"ds aggregation =\n{aggr_ds_query}")
print(f"ds aggregation =\n{aggr_ds_query.loc[target_date]}")
filtered = aggr_ds_query.loc[target_date, :]
print(f'ds for {target_date} = {filtered}')

ds aggregation =
                     pnl           vega
AsOfDate                               
2018-12-18   9603.209285   17851.328543
2018-12-19  18303.188058 -108762.794219
2018-12-20  12123.513677  -92587.232258
ds aggregation =
pnl      18303.188058
vega   -108762.794219
Name: 2018-12-19, dtype: float64
ds for 2018-12-19 = pnl      18303.188058
vega   -108762.794219
Name: 2018-12-19, dtype: float64


In [ ]:
mdx_query = connector.mdx_query("""
SELECT 
NON EMPTY {[Measures].[pnl.SUM],[Measures].[vega.SUM]} ON COLUMNS, 
NON EMPTY [Time].[HistoricalDates].[AsOfDate].Members ON ROWS 
FROM [EquityDerivativesCube]
""")

In [31]:
print(f'mdx query : {mdx_query.dataframe}')

TypeError: unsupported operand type(s) for //: 'int' and 'NoneType'